In [52]:
import networkx as nx
import random
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import gower 
import dill
from pathos.multiprocessing import ProcessingPool

path = os.getcwd()
path
rootpath = '/Users/rtseinstein/Documents/GitHub/Solar-Adoption-Agent-based-Model/'

survey = pd.read_csv(rootpath+'data/survey/gps_final_scored_phase3.csv')
survey['PEOPLE_TOT_3PLUS']=survey['PEOPLE_TOT_3PLUS'].fillna(0)
#survey.info()

#read in the main households data
households = pd.read_csv(rootpath+'data/households_main/households_main.csv')
households = households.drop(columns='Unnamed: 0')

demographics = survey[['CASE_ID','AGE_BINNED','INCOME_BINNED','PEOPLE_TOT_3PLUS']].set_index('CASE_ID')
tpb_attributes = survey[['CASE_ID','attitude','subnorms','pbc']].set_index('CASE_ID')

## need to map the numbers to categories in the survey datatable
income_map= {1:'less75k',2:'less75k',3:'75to100k',4:'100to150k',5:'150kplus'}
age_map = {1.:'25to44',2.:'45to54',3.:'55to64',4.:'65plus'}
#need to do this mapping in the households data
size_map={'1person':0.,'2person':0.,'3person':np.random.choice([0.,1.]),'4person':1.,'5person':1.,'6person':1.,'7plus':1.}

demographics['INCOME_BINNED']=demographics['INCOME_BINNED'].map(income_map)
demographics['AGE_BINNED']=demographics['AGE_BINNED'].map(age_map)

households = households.dropna()
households.drop(households.tail(1).index,inplace=True) # drop last n rows

## add the required household row as last row in the demographics survey data
demographics = demographics.rename(columns={'AGE_BINNED':'age','INCOME_BINNED':'income','PEOPLE_TOT_3PLUS':'household_'})

filenames=[]
for i in range(24):
    filenames.append('df_'+str(i+1)+'.csv')


#definition of initialize function 
def initialize(df,df_name):
    print(f'beginning initialization of {df_name}')
    attitude={}
    subnorms={}
    pbc={}
    errorids = []
    for household_case in list(df.index):
        new_record = df.loc[household_case]

        #add to last row of demographics data
        demo = demographics
        demo['household_'] = demo['household_'].astype('object')
        demo = demo.append(new_record)

        #calculate gower's distance
        new_row = gower.gower_matrix(demo)[-1]
        #new_row

        indexes = list(demo.index)

        temp = min(new_row)
        res = [i for i, j in enumerate(new_row) if j == temp]
        #res

        caseids = []
        for i in res:
            caseids.append(indexes[i])


        c = []
        for i in caseids:
            c.append(int(i))

        atts = []
        sn = []
        p = []
        try:
            for i in c:
                if i!=c[-1]:
                    atts.append(tpb_attributes.loc[i]['attitude'])
                    sn.append(tpb_attributes.loc[i]['subnorms'])
                    p.append(tpb_attributes.loc[i]['pbc'])
            
            attitude[household_case]= np.random.choice(atts)
            subnorms[household_case]=np.random.choice(sn)
            pbc[household_case]=np.random.choice(p)
        except:
            attitude[household_case]= 0
            subnorms[household_case]=0
            pbc[household_case]=0
            print(household_case)
            errorids.append(household_case)
            continue
    
    df['attitude']= list(attitude.values())
    df['subnorms']= list(subnorms.values())
    df['pbc']= list(pbc.values())

    df.to_csv(rootpath+f'data/initialization_subsets_24/{df_name}_initialized.csv')
    print(f'finished exporting the initialized file of {df_name}')

pool = ProcessingPool()
results = pool.map(initialize(,))



<class 'pandas.core.frame.DataFrame'>
Int64Index: 80280 entries, 0 to 80279
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rand_point  80280 non-null  float64
 1   case_id     80280 non-null  int64  
 2   bgid        80280 non-null  int64  
 3   age         80280 non-null  object 
 4   income      80280 non-null  object 
 5   sample_inc  80280 non-null  float64
 6   ami_catego  80280 non-null  int64  
 7   tract       80280 non-null  int64  
 8   elec_consu  80280 non-null  float64
 9   TRACTCE10   80280 non-null  float64
 10  BLOCKCE10   80280 non-null  float64
 11  GEOID10     80280 non-null  float64
 12  geometry    80280 non-null  object 
 13  household_  80280 non-null  int64  
dtypes: float64(6), int64(5), object(3)
memory usage: 9.2+ MB


In [53]:
household_demographics= households[['case_id','age','income','household_']]
new_caseids=[]
for i in list(household_demographics['case_id']):
    new_caseids.append(str(i).zfill(9))

household_demographics['case_id']= new_caseids
household_demographics['household_']=household_demographics['household_'].astype('object')
household_demographics= household_demographics.rename(columns={'case_id':'CASE_ID'})
household_demographics = household_demographics.set_index('CASE_ID')


household_demographics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80280 entries, 000033813 to 000061348
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   age         80280 non-null  object
 1   income      80280 non-null  object
 2   household_  80280 non-null  object
dtypes: object(3)
memory usage: 2.4+ MB
ipykernel_launcher:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [51]:
## separate the main file into several subsets of 3345 each: 24 files 
for i in list(range(24)):
    df= np.array_split(household_demographics, 24)[i]
    #df.to_csv(rootpath+f'data/initialization_subsets_24/df_{i+1}.csv')


In [54]:
# for each file, read and call the function to initialize
#need: demographics file and tpb_attributes file of the survey data 


df_sample = df.head(10) 
initialize(df_sample,'df_sample_')

ipykernel_launcher:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['df_1.csv',
 'df_2.csv',
 'df_3.csv',
 'df_4.csv',
 'df_5.csv',
 'df_6.csv',
 'df_7.csv',
 'df_8.csv',
 'df_9.csv',
 'df_10.csv',
 'df_11.csv',
 'df_12.csv',
 'df_13.csv',
 'df_14.csv',
 'df_15.csv',
 'df_16.csv',
 'df_17.csv',
 'df_18.csv',
 'df_19.csv',
 'df_20.csv',
 'df_21.csv',
 'df_22.csv',
 'df_23.csv',
 'df_24.csv']

In [74]:
import glob 
filenames= glob.glob(rootpath+'data/initialization_subsets_24/*.csv')

In [79]:
filenames[0][100:-4]

'df_14'